## 데이터 셋을 섞어야 한다.

* 용어
- clean : original CIFAR10
- adv : adversarial CIFAR10
- aug : augmented CIFAR10

* 방법
1. clean(BASELINE)
2. clean + adv -> clean : adv = [(90:10), (80:20), (70:30), (60:40)]
3. clean + aug -> clean : aug = [(90:10), (80:20), (70:30), (60:40)]
4. clean + adv + aug -> 미정

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms, utils
import torchvision.models as models

import os, random
from tqdm import tqdm
from PIL import Image
import numpy as np

from options.train_options import TrainOptions
from networks.LeNet import LeNet
from utils.Logger import Logger

## Load clean examples(original datasets)

### clean test set

In [6]:
clean_test_examples = []

clean_test_dataset = datasets.CIFAR10(
    root='./datasets/cifar10/',
    train=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((.5, .5, .5), (.5, .5, .5))
    ])
)

clean_test_loader = torch.utils.data.DataLoader(
    dataset=clean_test_dataset, batch_size=1, shuffle=True, num_workers=2
)

for (data, label) in tqdm(clean_test_loader):
    clean_test_examples.append((data, label))

# for (data, label) in clean_test_examples:
#     print(label)
#     print(data)
#     break


100%|██████████| 10000/10000 [00:12<00:00, 822.37it/s]
tensor([2])
tensor([[[[-0.1451, -0.1529, -0.1373,  ..., -0.0353, -0.0431, -0.0353],
          [-0.1451, -0.1451, -0.1373,  ..., -0.0118, -0.0275, -0.0039],
          [-0.1216, -0.1216, -0.1137,  ...,  0.0196,  0.0118,  0.0353],
          ...,
          [-0.0980, -0.1059, -0.1059,  ...,  0.1686,  0.1922,  0.0824],
          [-0.1216, -0.1216, -0.1059,  ...,  0.3725,  0.0980, -0.0824],
          [-0.0980, -0.0980, -0.0824,  ..., -0.0431, -0.3333, -0.3020]],

         [[-0.0431, -0.0510, -0.0431,  ..., -0.0196, -0.0275, -0.0118],
          [-0.0353, -0.0431, -0.0353,  ...,  0.0039, -0.0118,  0.0118],
          [-0.0196, -0.0196, -0.0118,  ...,  0.0353,  0.0275,  0.0510],
          ...,
          [ 0.0275,  0.0039, -0.0118,  ...,  0.2157,  0.2392,  0.1373],
          [ 0.0039, -0.0118, -0.0118,  ...,  0.4118,  0.1294, -0.0588],
          [ 0.0039, -0.0039, -0.0039,  ..., -0.0196, -0.3098, -0.2941]],

         [[-0.5765, -0.5843, -0.576

## Load adversarial examples
### get adversarial examples
- adv_examples : 적대적 예시 리스트 [(data, labels), ..., ]
  - data : 이미지가 ToTensor()에 의해 tensor화 된 것
  - labels : 이미지에 대한 ground truth, Tensor([라벨])

### adversary test set

In [7]:
tf = transforms.ToTensor()

ADV_TEST_DIR = './adversarial_examples/CIFAR10/test/'

# label
classes = os.listdir(ADV_TEST_DIR + '0.05/')

adv_test_examples = []

i = 0
for _class in classes:
    # print('\n' + _class, ' processing')
    # images
    train_images = os.listdir(ADV_TEST_DIR + '0.05/' + _class + '/')
    for image in tqdm(train_images):
        data = Image.open(ADV_TEST_DIR + '0.05/' + _class + '/' + image)
        data = tf(data)

        label = torch.tensor(np.array([int(_class)]))
        adv_test_examples.append( (data, label) )

# for shuffling check
# print(adv_test_examples[0][1])
# print(adv_test_examples[9_999][1])

# load data loader
# adv_test_loader = torch.utils.data.DataLoader(
#     dataset=adv_test_examples, batch_size=1, shuffle=True, num_workers=2
# )

# for (data, label) in adv_test_examples:
#     print(label)
#     print(data)
#     break

100%|██████████| 1000/1000 [00:00<00:00, 3737.63it/s]tensor([8])
tensor([[[0.4314, 0.4941, 0.5137,  ..., 0.4314, 0.3922, 0.4235],
         [0.4863, 0.5255, 0.5333,  ..., 0.4314, 0.3882, 0.4118],
         [0.4863, 0.5020, 0.4863,  ..., 0.4392, 0.3882, 0.3961],
         ...,
         [0.0118, 0.0118, 0.0314,  ..., 0.0235, 0.0000, 0.1216],
         [0.0000, 0.0000, 0.0314,  ..., 0.0000, 0.0000, 0.0824],
         [0.0118, 0.0000, 0.0000,  ..., 0.0000, 0.0078, 0.0706]],

        [[0.5843, 0.6471, 0.6588,  ..., 0.5961, 0.5569, 0.5882],
         [0.6431, 0.6824, 0.6824,  ..., 0.5961, 0.5529, 0.5765],
         [0.6431, 0.6588, 0.6353,  ..., 0.6039, 0.5529, 0.5647],
         ...,
         [0.1529, 0.1529, 0.1725,  ..., 0.0784, 0.0314, 0.1608],
         [0.1176, 0.1255, 0.1569,  ..., 0.0549, 0.0431, 0.1255],
         [0.1294, 0.0980, 0.0980,  ..., 0.0471, 0.0627, 0.1176]],

        [[0.6431, 0.7059, 0.7216,  ..., 0.6510, 0.6118, 0.6431],
         [0.6902, 0.7294, 0.7333,  ..., 0.6588, 0.6078, 0.

### shuffle the adversarial examples
- 라벨 별로 들어 있는 적대적 예시 배열을 랜덤하게 섞는다.

In [ ]:
# for testing
random.shuffle(adv_test_examples)

print(adv_test_examples[0][1])
print(adv_test_examples[9_999][1])

### create mixed datasets
- clean의 일부와 adversary의 일부를 +로 합친다.
    ```
    clean+adv = clean[:int(len(clean) * 0.8)] + adv[:(int)len(adv) * 0.2]
    shuffle clean + adv
    ```
- 섞은 것을 학습에 활용한다.

In [ ]:
# mix test
mixed_test_dataset = clean_test_examples[:int(len(clean_test_examples) * 0.8)] + adv_test_examples[:int(len(adv_test_examples) * 0.2)]
# random.shuffle(mixed_test_dataset)

### make mixed data loader

In [ ]:
# testing
mixed_test_loader = torch.utils.data.DataLoader(
    mixed_test_dataset, batch_size=64, shuffle=True, num_workers=2
)

## Train
- 새로 합성한 데이터 셋을 활용해서 학습 진행
- TODO :: data loader 두 개 섞기

In [ ]:
USE_CUDA = torch.cuda.is_available()

if __name__ == '__main__':

    DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
    print('Training will be activated in', DEVICE)

    NETWORK = 'lenet'
    # DATASET = opt.dataset
    SAVE_DIR = './models/LeNet/'
    NUM_CLASSES = 10

    IS_PRETRAINED = False
    IS_TRANSFERED = False

    EPOCHS = 30
    BATCH_SIZE = 64
    LEARNING_RATE = 0.001
    MOMENTUM = 0.9

    CRITERION = 'crossentropy'
    OPTIMIZER = 'sgd'

    logger = Logger(model=NETWORK, dataset='adversary cifar10')
    hyper_parameter_infos = """\
    - is_pretrained : %s
    - is_transfered : %s
    - epochs : %s
    - batch_size : %s
    - learning_rate : %s
    - momentum : %s
    - criterion : %s
    - optimizer : %s\
    """ % (IS_PRETRAINED, IS_TRANSFERED, EPOCHS, BATCH_SIZE, LEARNING_RATE, MOMENTUM, CRITERION, OPTIMIZER)
    logs = ''

    # models : lenet5, resnet18, vgg16
    if NETWORK == 'lenet':
        model = LeNet(num_classes=NUM_CLASSES)
    if NETWORK == 'resnet18':
        model = models.resnet18(pretrained=IS_PRETRAINED,
                                num_classes=NUM_CLASSES)
    if NETWORK == 'vgg16':
        model = models.vgg16(pretrained=IS_PRETRAINED, num_classes=NUM_CLASSES)

    print(NETWORK + ' will be used')
    print(hyper_parameter_infos)
    model.to(DEVICE)

    if CRITERION == 'crossentropy':
        criterion = nn.CrossEntropyLoss()
    if CRITERION == 'mseloss':
        criterion = nn.MSELoss()

    if OPTIMIZER == 'sgd':
        optimizer = optim.SGD(model.parameters(),
                              lr=LEARNING_RATE, momentum=MOMENTUM)
    if OPTIMIZER == 'adam':
        optimizer = optim.Adam(
            model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

    model.train()

    for epoch in range(EPOCHS):
        # print('%d epoch started' % epoch)
        running_loss = .0
        for _, (data, target) in enumerate(mixed_train_loader):
            data[0], target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()

            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('%d epoch loss : %.3f' %
              (epoch + 1, running_loss / len(mixed_train_loader)))
        logs += '%d epoch loss : %.3f' % (epoch + 1,
                                          running_loss / len(mixed_train_loader)) + '\n'

    # use last epoch only
    torch.save(model.state_dict(), SAVE_DIR + NETWORK + '_adversary_cifar10' + '_net.pth')
    logger.create_txt(hyper_parameter_infos, logs)
